# Data Pre-Processing

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import seaborn as sns
from matplotlib import pyplot as plt
from dateutil import parser
import datetime

## Timeseries

In [ ]:
us = pd.read_csv("https://github.com/nytimes/covid-19-data/blob/master/us.csv?raw=true", parse_dates=['date'])
us_states = pd.read_csv("https://github.com/nytimes/covid-19-data/blob/master/us-states.csv?raw=true", parse_dates=['date'])
us

In [ ]:
us_diff = us.set_index('date')
assert us_diff.resample('1d').asfreq().shape == us_diff.shape
us_diff -= us_diff.shift(1)
us_diff.iloc[0, :] = us.iloc[0, 1:]
us_diff

In [ ]:
fig, ax = plt.subplots()

sns.lineplot(data=us_diff.rolling('14d').mean(), x='date', y='cases')

fig.set_size_inches(12,8)
plt.show()

## Fitting everything on one line

### Anonymous Functions

When you're transforming data, you'll often want to do something fairly simple that doesn't have an immediate pre-built function. If you're going to reuse it often, it might be worth defining a function and then applying it:

```python
def bool_to_yn(boolean):
  if boolean:
    return "Yes"
  else:
    return "No"
```

On the other hand, it may be something you'll only use once. If I want to do the opposite transformation (True to "Yes" and False to "No"), it's just extra effort and clutter. In that case, you can just do everything on one line:

```python
df['Infection Post-Boost'] = df['Infection Pre-Boost'].apply(lambda pre_boost: "No" if pre_boost else "Yes")
```

There are two things that are unusual here: the lambda syntax, and the one-line if statement. This is equivalent to:

```python
def pre_to_post(pre_boost):
    if pre_boost:
        return "No"
    else:
        return "Yes"

df['Infection Post-Boost'] = df['Infection Pre-Boost'].apply(pre_to_post)
```

Instead of naming each function, every function is single-use and gets the same name (lambda, for arcane reasons). The one-line `if` statement reads like English.

### List Comprehensions

Just like if statements, you can also iterate through, filter, and transform on one line. This is easier to show in an actual code cell:


In [ ]:
fruits = ['Honeyberry','Huckleberry','Jabuticaba','Jackfruit','Jambul','Japanese plum','Jostaberry','Jujube','Juniper berry','Kaffir Lime','Horned Melon','Kiwifruit','Kumquat','Lemon','Lime','Loganberry','Longan','Loquat','Lulo','Lychee','Magellan Barberry','Mamey Apple','Mamey Sapote','Mango','Mangosteen','Marionberry','Melon','Cantaloupe','Galia melon','Honeydew','Mouse melon','Musk melon','Watermelon','Miracle fruit','Momordica fruit','Monstera deliciosa','Mulberry','Nance','Nectarine']

multipart = [', '.join(fruit.split()[::-1]) for fruit in fruits if " " in fruit]

sorted(multipart)


In [ ]:
sorted(multipart, key=lambda val: val.lower())

List comprehensions let you apply a function to every element in a list (or a subset fulfilling certain conditions) all in one line, somewhat readably.

## Splitting one line across multiple

The reverse is also often useful. Pandas makes it very easy to do something called "method chaining." By default, functions on dataframes make and transform a copy of the dataframe instead of changing the existing dataframe. This lets you do something like this:

```python
df.sort_values('Sorting Column').drop_duplicates(subset=['Participant ID', 'Sample Pre-Infection'], keep='first').sort_values(by=['Participant ID','Days to Infection']).drop('Sorting Column', axis=1)
```

which is convenient but basically unreadable. There's sorting, removing duplicates, re-sorting, and cleaning columns, but it's hard to keep everything separate. Because Python cares about whitespace, you can't generally hit enter anywhere you like and move functions to a new line. However, you can generally start a new line in the middle of parentheses, square braces, and brackets. Since parentheses don't do anything, you're free to add them and do something like this:

```python
(df.sort_values('Sorting Column')
   .drop_duplicates(subset=['Participant ID', 'Sample Pre-Infection'], keep='first')
   .sort_values(by=['Participant ID','Days to Infection'])
   .drop('Sorting Column', axis=1))
```

This is a lot more legible, and if you don't need access to any of the intermediate states it's also cleaner. In general, it's nice to be able to put simple one-time transformations on one line, while splitting more complicated multi-part endeavours across multiple lines.

# Antibodies

A while back, we augmented sample test results (antibody levels in the form of AUC) in Excel. We can do the same stuff in Python:

In [ ]:
workbook_name = 'data/fake.xlsx'

In [ ]:
sample_info = pd.read_excel(workbook_name, sheet_name='Sample Information')
sample_info.head()

In [ ]:
participant_info = pd.read_excel(workbook_name, sheet_name='Participant Information')
participant_info.head()

In [ ]:
indexed = participant_info.set_index('Participant ID')
indexed.head()

In [ ]:
df = sample_info.join(indexed, on='Participant ID')
df.head()

In [ ]:
df['Infection Pre-Boost'] = df.apply(lambda row: row['Vax1 to Infection'] < row['Vax1 to Boost'] or pd.isna(row['Vax1 to Boost']), axis=1)
# Equivalently:
df['Infection Pre-Boost'] = (df['Vax1 to Infection'] < df['Vax1 to Boost']) | pd.isna(df['Vax1 to Boost'])
df['Infection Post-Boost'] = df['Infection Pre-Boost'].apply(lambda val: "No" if val else "Yes")
df['Days to Infection'] = df['Days to Vax1'] - df['Vax1 to Infection']
df['Days to Boost'] = df['Days to Vax1'] - df['Vax1 to Boost']
df['Sample Pre-Infection'] = df['Days to Infection'] <= 0

df.head()

### Side note

Applying a function to a series is pretty straightforward with Pandas. If the function you're applying depends on multiple values, you can access the whole row by applying it to the whole table:

```python
# Dataframe-level
df['Infection Pre-Boost'] = df.apply(lambda row: row['Vax1 to Infection'] < row['Vax1 to Boost'] or pd.isna(row['Vax1 to Boost']), axis=1)

# Series-level
df['Infection Post-Boost'] = df['Infection Pre-Boost'].apply(lambda val: "No" if val else "Yes")
```

You get to choose the name of the argument here, but I tend to use `row` to represent a row of a dataframe and `val` to represent a single value. You can probably do better than `val`, but at least it's not wrong.


In [ ]:
df['Sorting Column'] = abs(df['Days to Infection'] - 28)
df_dedup = (df.sort_values('Sorting Column')
              .drop_duplicates(subset=['Participant ID', 'Sample Pre-Infection'], keep='first')
              .sort_values(by=['Participant ID','Days to Infection'])
              .drop('Sorting Column', axis=1))
df_dedup['Log2AUC'] = np.log2(df['AUC'])
df_dedup.head()

In [ ]:
df_dedup.groupby('Participant ID').count().head()

In [ ]:
df_dedup.groupby('Participant ID').count().query('AUC == 1').head()

In [ ]:
insufficient = df_dedup.groupby('Participant ID').count().query('AUC == 1').index.to_numpy()
insufficient

In [ ]:
print("Unfiltered:", df_dedup.shape)
df_final = df_dedup[df_dedup['Participant ID'].apply(lambda val: val not in insufficient)]
print("Filtered:", df_final.shape)
df_final.head()

In [ ]:
df_final.pivot_table(values='Log2AUC', index='Sample Pre-Infection', columns='Participant ID', sort=False)

In [ ]:
import requests

pd.DataFrame(requests.get("https://api.fda.gov/device/covid19serology.json", params={'limit': 1000}).json()['results'])

In [ ]:
fda_data = requests.get('https://api.fda.gov/download.json').json()
for cat, result in fda_data['results'].items():
    for subcat, subresult in result.items():
        print(cat,":", subcat, ":", subresult['partitions'][-1]['file'])

In [ ]:
import io
import zipfile
import json

def get_fda(url):
    fname = url.split("/")[-1][:-4]
    json_data = zipfile.ZipFile(io.BytesIO(requests.get(url).content)).read(fname).decode()
    return pd.DataFrame(json.loads(json_data)['results'])

In [ ]:
test_df = get_fda("https://download.open.fda.gov/device/covid19serology/device-covid19serology-0001-of-0001.json.zip")
test_df.head()

In [ ]:
test_df.describe()

In [ ]:
test_df.dtypes

In [ ]:
titer_columns = [col for col in test_df.columns if 'titer' in col]
for col in titer_columns:
    test_df[col] = test_df[col].astype(int)
test_df.describe()

In [ ]:
test_df.groupby('device').count().head()

In [ ]:
print(len(test_df['device'].unique()))

In [ ]:
116 * 110

In [ ]:
test_df.groupby('device').count().groupby('igg_agree').count()

In [ ]:
test_count = test_df.groupby('device').count()
test_count[test_count['sample_id'] == 220]

In [ ]:
test_df.groupby('sample_id').count()

In [ ]:
print(test_df.groupby('sample_id').count().shape)
print(test_df.groupby(['sample_id'] + titer_columns).count().shape)

In [ ]:
sns.pairplot(test_df.drop_duplicates('sample_id').loc[:, titer_columns])

In [ ]:
one_per = np.log2(test_df.drop_duplicates('sample_id').loc[:, titer_columns])
sns.pairplot(one_per)

In [ ]:
rng = np.random.default_rng()
sns.pairplot(one_per + rng.normal(0, 0.3, one_per.shape))

In [ ]:
for_cluster = test_df.pivot_table(values='control', index='sample_id', columns='device', aggfunc='count', fill_value=0)
g = sns.clustermap(for_cluster)
plt.show()

## Practice

In [ ]:
candy = pd.read_csv('https://github.com/phoebewong/candy-hierarchy-2017/blob/master/candyhierarchy2017.csv?raw=true', encoding = "ISO-8859-1")
print(candy.shape)
candy.head()